In [1]:
import pandas as pd
PATH_SUBJ_EVAL = "../0_data/9_evaluation/Masterarbeit MusikAI Umfrage.csv"

In [2]:
df_all = pd.read_csv(PATH_SUBJ_EVAL)
df_all.head()

,Zeitstempel,Video 1: Welcher Song gefällt Ihnen besser?,Video 2: Welcher Song gefällt Ihnen besser?,Video 3: Welcher Song gefällt Ihnen besser?,Video 4: Welcher Song gefällt Ihnen besser?,Video 5: Welcher Song gefällt Ihnen besser?,Video 6: Welcher Song gefällt Ihnen besser?,Video 7: Welcher Song gefällt Ihnen besser?,Video 8: Welcher Song gefällt Ihnen besser?,Video 9: Welcher Song gefällt Ihnen besser?,Video 10: Welcher Song gefällt Ihnen besser?,"War es schwierig für Sie, sich für einen Song zu entscheiden oder war die Wahl immer klar?",Ist Ihnen Musik wichtig?,Haben Sie eine musikalische Affinität?,Spielen Sie ein Instrument?,Welchem Geschlecht fühlen Sie sich zugehörig?,Wie alt sind Sie?
0,2023/05/03 11:40:32 PM OESZ,Song B,Song A,Song A,Song B,Song B,Song A,Song B,Song A,Song B,Song B,3,NaN,NaN,NaN,männlich,18-25
1,2023/05/09 11:57:50 AM OESZ,Song A,Song A,Song B,Song B,Song B,Song B,Song B,Song B,Song A,Song B,2,sehr wichtig,ja,"ja, hobbymäßig",männlich,25-35
2,2023/05/09 12:31:35 PM OESZ,Song B,Song A,Song B,Song A,Song B,Song B,Song A,Song B,Song A,Song A,4,normal,nein,nein,weiblich,18-25
3,2023/05/09 12:32:27 PM OESZ,Song B,Song A,Song A,Song B,Song A,Song A,Song B,Song B,Song B,Song B,4,sehr wichtig,ja,nein,männlich,18-25
4,2023/05/09 12:37:31 PM OESZ,Song B,Song A,Song B,Song B,Song B,Song B,Song B,Song B,Song B,Song B,4,normal,ja,"ja, hobbymäßig",weiblich,18-25


In [3]:
df_cleaned = df_all[df_all["Video 7: Welcher Song gefällt Ihnen besser?"] == "Song B"]
df_cleaned.head()

,Zeitstempel,Video 1: Welcher Song gefällt Ihnen besser?,Video 2: Welcher Song gefällt Ihnen besser?,Video 3: Welcher Song gefällt Ihnen besser?,Video 4: Welcher Song gefällt Ihnen besser?,Video 5: Welcher Song gefällt Ihnen besser?,Video 6: Welcher Song gefällt Ihnen besser?,Video 7: Welcher Song gefällt Ihnen besser?,Video 8: Welcher Song gefällt Ihnen besser?,Video 9: Welcher Song gefällt Ihnen besser?,Video 10: Welcher Song gefällt Ihnen besser?,"War es schwierig für Sie, sich für einen Song zu entscheiden oder war die Wahl immer klar?",Ist Ihnen Musik wichtig?,Haben Sie eine musikalische Affinität?,Spielen Sie ein Instrument?,Welchem Geschlecht fühlen Sie sich zugehörig?,Wie alt sind Sie?
0,2023/05/03 11:40:32 PM OESZ,Song B,Song A,Song A,Song B,Song B,Song A,Song B,Song A,Song B,Song B,3,NaN,NaN,NaN,männlich,18-25
1,2023/05/09 11:57:50 AM OESZ,Song A,Song A,Song B,Song B,Song B,Song B,Song B,Song B,Song A,Song B,2,sehr wichtig,ja,"ja, hobbymäßig",männlich,25-35
3,2023/05/09 12:32:27 PM OESZ,Song B,Song A,Song A,Song B,Song A,Song A,Song B,Song B,Song B,Song B,4,sehr wichtig,ja,nein,männlich,18-25
4,2023/05/09 12:37:31 PM OESZ,Song B,Song A,Song B,Song B,Song B,Song B,Song B,Song B,Song B,Song B,4,normal,ja,"ja, hobbymäßig",weiblich,18-25
5,2023/05/09 12:40:17 PM OESZ,Song B,Song A,Song A,Song B,Song B,Song A,Song B,Song B,Song B,Song A,3,sehr wichtig,nein,nein,männlich,25-35


In [4]:
survey_order = {
    "orig vs. a1": [
        {"question": 2, "orig": "Song A", "a1": "Song B"},
        {"question": 10, "orig": "Song B", "a1": "Song A"},
        {"question": 4, "orig": "Song A", "a1": "Song B"}
    ],
    "orig vs. d": [
        {"question": 5, "orig": "Song A", "d": "Song B"},
        {"question": 9, "orig": "Song A", "d": "Song B"},
        {"question": 8, "orig": "Song B", "d": "Song A"}
    ],
    "a1 vs. d": [
        {"question": 3, "a1": "Song A", "d": "Song B"},
        {"question": 6, "a1": "Song B", "d": "Song A"},
        {"question": 1, "a1": "Song B", "d": "Song A"}
    ]
}

In [8]:
def get_results(df, survey_order):
    survey_results = {}

    for comparison in survey_order:

        comp1 = (comparison.split(" vs. "))[0]
        comp2 = (comparison.split(" vs. "))[1]
        survey_results[comparison] = {}

        for i,q in enumerate(survey_order[comparison]):
            survey_results[comparison].update({
                f"{comp1}_v{i+1}": df[f"Video {q['question']}: Welcher Song gefällt Ihnen besser?"].value_counts()[q[comp1]],
                f"{comp2}_v{i+1}": df[f"Video {q['question']}: Welcher Song gefällt Ihnen besser?"].value_counts()[q[comp2]],
            })
        
        survey_results[comparison].update({
            f"{comp1}_all": survey_results[comparison][f"{comp1}_v1"] + survey_results[comparison][f"{comp1}_v2"] + survey_results[comparison][f"{comp1}_v3"],
            f"{comp2}_all": survey_results[comparison][f"{comp2}_v1"] + survey_results[comparison][f"{comp2}_v2"] + survey_results[comparison][f"{comp2}_v3"]
        })

    return survey_results

In [9]:
results_all = get_results(df_all, survey_order)
results_cleaned = get_results(df_cleaned, survey_order)

In [10]:
results_all


{'orig vs. a1': {'orig_v1': 40,
  'a1_v1': 16,
  'orig_v2': 35,
  'a1_v2': 21,
  'orig_v3': 24,
  'a1_v3': 32,
  'orig_all': 99,
  'a1_all': 69},
 'orig vs. d': {'orig_v1': 10,
  'd_v1': 46,
  'orig_v2': 36,
  'd_v2': 20,
  'orig_v3': 41,
  'd_v3': 15,
  'orig_all': 87,
  'd_all': 81},
 'a1 vs. d': {'a1_v1': 28,
  'd_v1': 28,
  'a1_v2': 24,
  'd_v2': 32,
  'a1_v3': 29,
  'd_v3': 27,
  'a1_all': 81,
  'd_all': 87}}

In [11]:
results_cleaned

{'orig vs. a1': {'orig_v1': 35,
  'a1_v1': 11,
  'orig_v2': 30,
  'a1_v2': 16,
  'orig_v3': 19,
  'a1_v3': 27,
  'orig_all': 84,
  'a1_all': 54},
 'orig vs. d': {'orig_v1': 6,
  'd_v1': 40,
  'orig_v2': 31,
  'd_v2': 15,
  'orig_v3': 36,
  'd_v3': 10,
  'orig_all': 73,
  'd_all': 65},
 'a1 vs. d': {'a1_v1': 24,
  'd_v1': 22,
  'a1_v2': 16,
  'd_v2': 30,
  'a1_v3': 24,
  'd_v3': 22,
  'a1_all': 64,
  'd_all': 74}}